In [1]:
import polars as pl

In [3]:
df = pl.read_parquet('recommendations.parquet/part-00000-5cb623b6-c0c3-43a3-bf7f-cbbc582b3f27-c000.snappy.parquet')

In [6]:
df.shape

(44776, 2)

In [7]:
df.head()

user_id,recommendations
i32,list[struct[2]]
13,"[{6405,6.312828}, {6407,6.287482}, … {6403,5.781521}]"
16,"[{637,5.115455}, {7129,5.015377}, … {442,4.401185}]"
22,"[{1313,4.699243}, {1067,4.193136}, … {938,3.918814}]"
26,"[{1200,4.117683}, {7972,4.000442}, … {1212,3.852738}]"
31,"[{7106,5.652311}, {7105,5.526443}, … {19891,5.07172}]"


In [40]:
df = df.explode('recommendations')

# Convert the struct elements to separate columns
df = df.with_columns([
    pl.col("recommendations").apply(lambda x: x["book_id"]).alias("book_id"),
    pl.col("recommendations").apply(lambda x: x["rating"]).alias("score")
]).drop("recommendations")

/var/folders/cc/01vgsm2x5mx1t9pzx9p0x9lh0000gn/T/ipykernel_88127/3440394774.py:5: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("recommendations").apply(lambda x: x["book_id"]).alias("book_id"),
/var/folders/cc/01vgsm2x5mx1t9pzx9p0x9lh0000gn/T/ipykernel_88127/3440394774.py:6: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("recommendations").apply(lambda x: x["rating"]).alias("score")
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [41]:
df

user_id,book_id,score
i32,i64,f64
13,6405,6.312828
13,6407,6.287482
13,6401,6.070915
13,6402,6.01066
13,6400,5.928829
…,…,…
875909,2584,4.349367
875909,821,4.298105
875909,2588,4.29718


In [8]:
all_books = pl.read_csv('data/interactions_top_books.csv')

In [62]:
def show_overlap(user_id):
    user_predictions = df.filter(pl.col('user_id')==user_id)
    return all_books.filter((pl.col('user_id')==user_id) & (pl.col('book_id').is_in(user_predictions.select('book_id'))))

In [49]:
test_users = pl.read_csv('als_user_ids.csv/part-00000-1d171945-078f-4ef9-9193-f26263f2f392-c000.csv')

In [60]:
test_users.sample(10)

user_id
i64
81755
165157
331138
85384
348609
202456
187188
180817
338198


In [129]:
relavent = 0
sample_size = 50
reccomended = 10*sample_size
for x in test_users.sample(sample_size).select('user_id').to_numpy():
    relavent += len(show_overlap(x))
print(relavent/reccomended)

0.366


In [75]:
books = pl.read_csv('data/books.csv', ignore_errors=True)

In [77]:
books = books.select('book_id', 'title')

In [85]:
def get_title(book_id):
    return books.filter(pl.col('book_id')==book_id).select('title').to_numpy()

In [87]:
get_title(81755)[0, 0]

'Signs and Cities: Black Literary Postmodernism'

In [88]:
show_overlap(81755)

user_id,book_id,is_read,rating,is_reviewed,shelved
i64,i64,i64,i64,i64,i64
81755,7389,1,5,0,1
81755,7391,1,5,0,1
81755,7387,1,5,0,1
81755,7169,1,5,0,1
81755,17305,1,4,0,1
81755,7388,1,4,0,1
81755,7421,1,4,0,1


In [127]:
user = test_users.sample(1).to_numpy()[0, 0]
for x in show_overlap(user).select('book_id').to_numpy():
    print(get_title(x))

[['Marriage Special Package- 2 books- Five Love Languages/Fight Fair']]
[['Love As A Foreign Language #5']]
[]
[['How to Make Anyone Fall in Love with You']]
[]
[['The Da Vinci Code (Robert Langdon, #2)']]
[]


In [128]:
show_overlap(user)

user_id,book_id,is_read,rating,is_reviewed,shelved
i64,i64,i64,i64,i64,i64
422939,943,1,5,0,1
422939,941,1,5,0,1
422939,613,1,5,0,1
422939,938,1,5,0,1
422939,944,1,5,0,1
422939,968,1,5,0,1
422939,939,1,5,0,1
